In [2]:
# Data Cleaning
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


#vg_sales = pd.read_csv('/Users/melinda_zxy/Desktop/EMORY/senior/QTM 347/vgsales.csv')
vg_sales = pd.read_csv('/Users/dell/Desktop/QTM_347/final/vgsales.csv')
vg_sales.head()
vg_sales.info()

duplicates = vg_sales.duplicated().sum()

# Handle missing values： “Year” and “Publisher”
vg_sales['Year'] = vg_sales['Year'].fillna(vg_sales['Year'].median())
vg_sales['Publisher'] = vg_sales['Publisher'].fillna('Unknown')
vg_sales['Year'] = vg_sales['Year'].astype(int)

cleaned_summary = {
    "Duplicates Removed": duplicates,
    "Missing Values (Post-Cleaning)": vg_sales.isnull().sum().to_dict(),
}

descriptive_stats = vg_sales.describe()

print("Data Cleaning Summary:", cleaned_summary)
print("Descriptive Statistics:", descriptive_stats)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16598 entries, 0 to 16597
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Rank          16598 non-null  int64  
 1   Name          16598 non-null  object 
 2   Platform      16598 non-null  object 
 3   Year          16327 non-null  float64
 4   Genre         16598 non-null  object 
 5   Publisher     16540 non-null  object 
 6   NA_Sales      16598 non-null  float64
 7   EU_Sales      16598 non-null  float64
 8   JP_Sales      16598 non-null  float64
 9   Other_Sales   16598 non-null  float64
 10  Global_Sales  16598 non-null  float64
dtypes: float64(6), int64(1), object(4)
memory usage: 1.4+ MB
Data Cleaning Summary: {'Duplicates Removed': np.int64(0), 'Missing Values (Post-Cleaning)': {'Rank': 0, 'Name': 0, 'Platform': 0, 'Year': 0, 'Genre': 0, 'Publisher': 0, 'NA_Sales': 0, 'EU_Sales': 0, 'JP_Sales': 0, 'Other_Sales': 0, 'Global_Sales': 0}}
Descriptive Stati

Jessie note: I want to utilize the categorical data as well. To do that, We need to get rid of a lot of the publishers and apply one hot encoding

Notation dictionary:

vg_sales_new = df that selected the publishers -> use this for analytics
vg_sales_unwanted = df that contains the games that contain the less popular publishers, just for the sake of double checking
vg_sales_encoded = df that chose the top 10% publisher,and have been encoded
vg_sales_ready = all numerical columns, dropped the original categorical columns and rank and name -> use this for machine learning 


In [3]:
#To get an idea of the values of sales
sales_data = vg_sales[['NA_Sales','EU_Sales',"JP_Sales",'Other_Sales','Global_Sales']]

summary_table = sales_data.describe().transpose()

summary_table['Mode'] = sales_data.mode().iloc[0]  # Taking the first mode for each column

print(summary_table)

                count      mean       std   min   25%   50%   75%    max  Mode
NA_Sales      16598.0  0.264667  0.816683  0.00  0.00  0.08  0.24  41.49  0.00
EU_Sales      16598.0  0.146652  0.505351  0.00  0.00  0.02  0.11  29.02  0.00
JP_Sales      16598.0  0.077782  0.309291  0.00  0.00  0.00  0.04  10.22  0.00
Other_Sales   16598.0  0.048063  0.188588  0.00  0.00  0.01  0.04  10.57  0.00
Global_Sales  16598.0  0.537441  1.555028  0.01  0.06  0.17  0.47  82.74  0.02


In [4]:
unique_count = vg_sales['Publisher'].nunique()
print(f"Unique count for publisher ': {unique_count}")


Unique count for publisher ': 578


We need to convert categorical data into encoded columns. However, we have noticed that the publisher column has too many categories. For the sake of simplicity of the project. 

In [5]:
# Calculate frequency and percentage
freq_pub = vg_sales['Publisher'].value_counts().reset_index()
freq_pub.columns = ['Publisher', 'Frequency']
freq_pub['Percent'] = (freq_pub['Frequency'] / len(vg_sales)) * 100

freq_year = freq_pub.sort_values(by='Frequency', ascending=False).reset_index(drop=True)

print(freq_pub)


                        Publisher  Frequency   Percent
0                 Electronic Arts       1351  8.139535
1                      Activision        975  5.874202
2              Namco Bandai Games        932  5.615134
3                         Ubisoft        921  5.548861
4    Konami Digital Entertainment        832  5.012652
..                            ...        ...       ...
573                          Warp          1  0.006025
574                           New          1  0.006025
575                         Elite          1  0.006025
576               Evolution Games          1  0.006025
577             UIG Entertainment          1  0.006025

[578 rows x 3 columns]


In [6]:
filtered_freq_pub = freq_pub[(freq_pub['Frequency'] > 30) & (freq_pub['Frequency'] < 50)]

print(filtered_freq_pub)

             Publisher  Frequency   Percent
46   DTP Entertainment         45  0.271117
47      GT Interactive         45  0.271117
48           Destineer         45  0.271117
49           Alchemist         43  0.259067
50           MTV Games         41  0.247018
51               PQube         39  0.234968
52         Global Star         39  0.234968
53         Takara Tomy         37  0.222918
54     SouthPeak Games         37  0.222918
55               Spike         37  0.222918
56      TDK Mediactive         36  0.216894
57                 3DO         36  0.216894
58        Nordic Games         35  0.210869
59  BAM! Entertainment         35  0.210869
60    Black Bean Games         34  0.204844
61           Zoo Games         33  0.198819
62        Game Factory         32  0.192794
63           Psygnosis         32  0.192794
64           Mindscape         32  0.192794


In [7]:
mean_percent = freq_pub['Percent'].mean()
median_percent = freq_pub['Percent'].median()
percentile_90 = freq_pub['Percent'].quantile(0.90)

# Create a summary table
summary_table = pd.DataFrame({
    'Statistic': ['Mean', 'Median','Top 10% Percent'],
    'Percentage': [mean_percent, median_percent,percentile_90]
})
print(summary_table)


#top 75% is 0.06, 10 games

         Statistic  Percentage
0             Mean    0.173010
1           Median    0.018074
2  Top 10% Percent    0.212676


We will be using a cut-off threshold of 0.21%, which is the top10% publisher based on number of games they have produced. This also the selected larger publishers published more than 35 games.


In [8]:
selected_publisher = freq_pub[freq_pub['Percent'] > 0.21]
print(selected_publisher.shape)

unwanted_publisher=freq_pub[freq_pub['Percent']<=0.21]

(60, 3)


defining the rank categories

In [9]:
vg_sales["Rank"]
import pandas as pd

# Assuming vg_sales is the dataset and it contains the 'Rank' column

# Define the categories and their labels
rank_categories = ['Very High', 'High', 'Medium', 'Low', 'Very Low']

# Categorize 'Rank' into five categories based on quantiles
vg_sales['Rank_Category'] = pd.qcut(
    vg_sales['Rank'], 
    q=5, 
    labels=rank_categories
)

# Display the first few rows to verify the categorization
print(vg_sales[['Rank', 'Rank_Category']].head())
vg_sales["Rank_Category"]

   Rank Rank_Category
0     1     Very High
1     2     Very High
2     3     Very High
3     4     Very High
4     5     Very High


0        Very High
1        Very High
2        Very High
3        Very High
4        Very High
           ...    
16593     Very Low
16594     Very Low
16595     Very Low
16596     Very Low
16597     Very Low
Name: Rank_Category, Length: 16598, dtype: category
Categories (5, object): ['Very High' < 'High' < 'Medium' < 'Low' < 'Very Low']

In [10]:
vg_sales_new = vg_sales[vg_sales['Publisher'].isin(selected_publisher['Publisher'])]
vg_sales.shape # 16598,11
vg_sales_new.shape  # 13806,11
# 15449,11 when top 25%

(13806, 12)

In [11]:
# #check if there are very popular games in the unwanted publishers list  
# vg_sales_unwanted = vg_sales[~vg_sales['Publisher'].isin(selected_publisher['Publisher'])]
# vg_sales_unwanted_sorted = vg_sales_unwanted.sort_values(by='Global_Sales', ascending=False)
# print(vg_sales_unwanted_sorted.head)


In [12]:
# #pie chart by region  and publisher

# import matplotlib.pyplot as plt

# # Grouping data by Publisher and summing up sales columns
# sales_by_publisher = vg_sales.groupby('Publisher')[['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales']].sum()

# # Re-plotting with all data included but omitting titles for publishers with <5%
# for region in regions:
#     # Sorting by descending proportion for the current region
#     sorted_sales = sales_by_publisher[region].sort_values(ascending=False)
    
#     # Generate labels but omit the title for publishers contributing < 5%
#     total_sales = sorted_sales.sum()
#     labels = [
#         pub if sales / total_sales >= 0.05 else ""
#         for pub, sales in sorted_sales.items()
#     ]
    
#     # Plotting the pie chart
#     plt.figure(figsize=(6, 6))
#     plt.pie(
#         sorted_sales,
#         labels=labels,
#         autopct='%1.1f%%',
#         startangle=90
#     )
#     plt.title(f"{region} by Publisher")
#     plt.show()


In [13]:
from sklearn.preprocessing import OneHotEncoder


In [14]:
encoder = OneHotEncoder(sparse_output=False)  # Use `sparse_output=False` for an array output
onehotarray = encoder.fit_transform(vg_sales_new[["Publisher", "Genre", "Platform"]])

encoded_categories = []
for col, categories in zip(["Publisher", "Genre", "Platform"], encoder.categories_):
    encoded_categories.extend([f"{col}_{cat}" for cat in categories])

# Create a new DataFrame with the encoded columns
encoded_df = pd.DataFrame(onehotarray, columns=encoded_categories, index=vg_sales_new.index)

# Combine the original dataset with the encoded columns into a new dataset
vg_sales_encoded = pd.concat([vg_sales_new, encoded_df], axis=1)


print(vg_sales_encoded.columns)
#shape 13806,113

Index(['Rank', 'Name', 'Platform', 'Year', 'Genre', 'Publisher', 'NA_Sales',
       'EU_Sales', 'JP_Sales', 'Other_Sales',
       ...
       'Platform_SAT', 'Platform_SCD', 'Platform_SNES', 'Platform_TG16',
       'Platform_WS', 'Platform_Wii', 'Platform_WiiU', 'Platform_X360',
       'Platform_XB', 'Platform_XOne'],
      dtype='object', length=114)


In [15]:
print(list(vg_sales_encoded.columns))


['Rank', 'Name', 'Platform', 'Year', 'Genre', 'Publisher', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales', 'Rank_Category', 'Publisher_3DO', 'Publisher_505 Games', 'Publisher_5pb', 'Publisher_Acclaim Entertainment', 'Publisher_Activision', 'Publisher_Alchemist', 'Publisher_Atari', 'Publisher_Atlus', 'Publisher_BAM! Entertainment', 'Publisher_Banpresto', 'Publisher_Bethesda Softworks', 'Publisher_Capcom', 'Publisher_Codemasters', 'Publisher_Crave Entertainment', 'Publisher_D3Publisher', 'Publisher_DTP Entertainment', 'Publisher_Deep Silver', 'Publisher_Destineer', 'Publisher_Disney Interactive Studios', 'Publisher_Eidos Interactive', 'Publisher_Electronic Arts', 'Publisher_Empire Interactive', 'Publisher_Focus Home Interactive', 'Publisher_GT Interactive', 'Publisher_Global Star', 'Publisher_Hudson Soft', 'Publisher_Idea Factory', 'Publisher_Ignition Entertainment', 'Publisher_Infogrames', 'Publisher_Kadokawa Shoten', 'Publisher_Konami Digital Entertainment', 'Publish

In [16]:

columns_to_drop = ['Name', 'Platform', 'Publisher', 'Genre']
vg_sales_ready = vg_sales_encoded.drop(columns=columns_to_drop, errors='ignore')

#print(vg_sales_ready.columns)
print(vg_sales_ready)
#check all is numerical


        Rank  Year  NA_Sales  EU_Sales  JP_Sales  Other_Sales  Global_Sales  \
0          1  2006     41.49     29.02      3.77         8.46         82.74   
1          2  1985     29.08      3.58      6.81         0.77         40.24   
2          3  2008     15.85     12.88      3.79         3.31         35.82   
3          4  2009     15.75     11.01      3.28         2.96         33.00   
4          5  1996     11.27      8.89     10.22         1.00         31.37   
...      ...   ...       ...       ...       ...          ...           ...   
16588  16591  2008      0.01      0.00      0.00         0.00          0.01   
16591  16594  2004      0.01      0.00      0.00         0.00          0.01   
16592  16595  2008      0.01      0.00      0.00         0.00          0.01   
16594  16597  2003      0.01      0.00      0.00         0.00          0.01   
16595  16598  2008      0.00      0.00      0.00         0.00          0.01   

      Rank_Category  Publisher_3DO  Publisher_505 G

In [17]:
print(list(vg_sales_ready.columns))


['Rank', 'Year', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales', 'Rank_Category', 'Publisher_3DO', 'Publisher_505 Games', 'Publisher_5pb', 'Publisher_Acclaim Entertainment', 'Publisher_Activision', 'Publisher_Alchemist', 'Publisher_Atari', 'Publisher_Atlus', 'Publisher_BAM! Entertainment', 'Publisher_Banpresto', 'Publisher_Bethesda Softworks', 'Publisher_Capcom', 'Publisher_Codemasters', 'Publisher_Crave Entertainment', 'Publisher_D3Publisher', 'Publisher_DTP Entertainment', 'Publisher_Deep Silver', 'Publisher_Destineer', 'Publisher_Disney Interactive Studios', 'Publisher_Eidos Interactive', 'Publisher_Electronic Arts', 'Publisher_Empire Interactive', 'Publisher_Focus Home Interactive', 'Publisher_GT Interactive', 'Publisher_Global Star', 'Publisher_Hudson Soft', 'Publisher_Idea Factory', 'Publisher_Ignition Entertainment', 'Publisher_Infogrames', 'Publisher_Kadokawa Shoten', 'Publisher_Konami Digital Entertainment', 'Publisher_LucasArts', 'Publisher_MTV Games', 'Pub

In [18]:
vg_sales_ready = vg_sales_ready[vg_sales_ready['Global_Sales'] != 82.74]

In [19]:
all_numeric = vg_sales_ready.select_dtypes(include=['number']).shape[1] == vg_sales_ready.shape[1]

print(all_numeric)

False
